In [3]:
#imports
import torch
from torch import nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch
from torch.utils.data import TensorDataset
import ast
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn import metrics
from reg_sklearn import DataHandler
from econml.dml import LinearDML


import matplotlib.pyplot as plt


In [19]:
##load the data
patients = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/patients.csv')
organs = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/organs.csv')
outcomes = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/outcomes.csv')
outcomes_noiseless = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/outcomes_noiseless.csv')

outcome = 'eGFR'



outcomes = outcomes[['pat_id', 'org_id', outcome]]
outcomes_noiseless = outcomes_noiseless[['pat_id', 'org_id', outcome]]

outcomes = outcomes.dropna()
outcomes_noiseless = outcomes_noiseless.dropna()

outcomes = outcomes.iloc[:, 2]
outcomes = outcomes.to_numpy()

# CAUTION: Uncomment this line if loading data from a csv file
# self.outcomes = self.outcomes.map(ast.literal_eval)
# self.outcomes_noiseless = self.outcomes_noiseless.map(ast.literal_eval)


# if outcome == 'eGFR_3':
#     outcomes = self.outcomes.map(lambda x: x['eGFR'][2] if x and 'eGFR' in x else None)
#     outcomes_noiseless = self.outcomes_noiseless.map(lambda x: x['eGFR'][2] if x and 'eGFR' in x else None)

# else:
#     raise ValueError('Outcome not supported')



patients = pd.get_dummies(patients)
# organs = pd.get_dummies(organs) 

In [20]:
#cluster the organs treatment into 10 clusters
from kmodes.kprototypes import KPrototypes

# Assuming 'df' is your DataFrame
# Specify the columns of your categorical features
catColumnsPos = [organs.columns.get_loc(col) for col in list(organs.select_dtypes('object').columns)]

kproto = KPrototypes(n_clusters=10, init='Cao', verbose=2)
clusters = kproto.fit_predict(organs, categorical=catColumnsPos)





Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 130, ncost: 1415797.8254105325
Run: 1, iteration: 2/100, moves: 112, ncost: 1246401.0685256834
Run: 1, iteration: 3/100, moves: 85, ncost: 1154153.788482635
Run: 1, iteration: 4/100, moves: 71, ncost: 1098437.752427038
Run: 1, iteration: 5/100, moves: 54, ncost: 1062886.279838224
Run: 1, iteration: 6/100, moves: 47, ncost: 1039667.37181704
Run: 1, iteration: 7/100, moves: 40, ncost: 1023983.2776624114
Run: 1, iteration: 8/100, moves: 33, ncost: 1012464.3180342977
Run: 1, iteration: 9/100, moves: 18, ncost: 1007603.2392770991
Run: 1, iteration: 10/100, moves: 28, ncost: 1001194.0106350811
Run: 1, iteration: 11/100, moves: 16, ncost: 998519.8653065877
Run: 1, iteration: 12/100, moves: 7, ncost: 997391.9761385944
Run: 1, iteration: 13/100, moves: 14, ncost: 995233.6958689806
Run: 1, iteration: 14/100, m

In [22]:
#fit the model
from econml.dml import LinearDML
from sklearn.linear_model import LogisticRegressionCV


est = LinearDML(model_t=LogisticRegressionCV(), discrete_treatment=True)
est.fit(outcomes, clusters , X=patients, W=None)
point = est.const_marginal_effect(X=patients)
effect = est.effect(patients, T0=0, T1=1)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

In [24]:
effect.shape

(1000,)